### Générateur des phrase en 3D (11 and 13 -> 16)

In [ ]:
import csv
from pathlib import Path
from random import randint

data_path = Path.cwd().parent.parent / "data"

contrast_csv_dir = data_path / "11_lesion_contrast_csv_dir"
if not contrast_csv_dir.exists():
    raise FileNotFoundError(f"Le dossier {contrast_csv_dir} n'existe pas. Veuillez d'abord exécuter le script de contrast_calculation.ipynb")

lesion_info_csv_dir = data_path / "13_lesion_proportion_with_intensity_csv_dir"
if not lesion_info_csv_dir.exists():
    raise FileNotFoundError(f"Le dossier {lesion_info_csv_dir} n'existe pas. Veuillez d'abord exécuter le 2e script de intensity_extration_per_label.ipynb")

template_txt_dir_path = data_path / "16_3D_template_txt_dir"
if not template_txt_dir_path.exists():
    template_txt_dir_path.mkdir(parents=True, exist_ok=True)

In [10]:
%run "./clinical_parser.ipynb"


id = P1,
sex = F,
timepoint_number = 3,
timepoint_infos = {'T1': <patient_info.TimepointInfo object at 0x0000025174C3D5D0>, 'T2': <patient_info.TimepointInfo object at 0x0000025178BEF290>, 'T3': <patient_info.TimepointInfo object at 0x0000025178C46AD0>},


id = P2,
sex = M,
timepoint_number = 4,
timepoint_infos = {'T1': <patient_info.TimepointInfo object at 0x0000025178C45C10>, 'T2': <patient_info.TimepointInfo object at 0x0000025178C455D0>, 'T3': <patient_info.TimepointInfo object at 0x0000025178C46810>, 'T4': <patient_info.TimepointInfo object at 0x0000025178C45350>},


id = P3,
sex = F,
timepoint_number = 4,
timepoint_infos = {'T1': <patient_info.TimepointInfo object at 0x0000025178C47FD0>, 'T2': <patient_info.TimepointInfo object at 0x0000025178C45B50>, 'T3': <patient_info.TimepointInfo object at 0x0000025178C450D0>, 'T4': <patient_info.TimepointInfo object at 0x0000025178C47410>},


timepoint = T1,
age = 28.09,
ms_type = SMRR,
edss = 3.5,
lesion_number = 18,
lesion_volume = 20674

In [11]:
def read_csv_dict(csv_path):
    """Lit un CSV et renvoie une liste de dicts (chaque ligne)."""
    with csv_path.open("r", encoding="utf-8") as f:
        reader = csv.DictReader(f)
        return [row for row in reader]

In [13]:
"""
11 and 13 -> 16
Générer les templates sous forme de {patient_id}_{timepoint}_{modalité}_{lesion_id}_3D_Description
"""

# Parcours des fichiers CSV de lésion
for file_path in lesion_info_csv_dir.iterdir():
    if not file_path.name.endswith("_lesion_proportion.csv"):
        continue

    parts = file_path.stem.split("_")
    if len(parts) == 5:
        patient_id, timepoint, modality = parts[0], parts[1], parts[2]
    elif len(parts) == 4:
        patient_id, timepoint, modality = parts[0], "T1", parts[1]
    else:
        print(f"[SKIP] Nom de fichier non conforme : {file_path.name}")
        continue

    lesion_rows = read_csv_dict(file_path)

    contrast_csv_path = contrast_csv_dir / f"{patient_id}_{timepoint}_{modality}_lesion_contrast.csv"
    contrast_rows = read_csv_dict(contrast_csv_path) if contrast_csv_path.exists() else []

    # Récupérer infos patient
    current_patient = next((p for p in patients if p.id == patient_id), None)
    if current_patient is None:
        raise ValueError(f"Patient {patient_id} non trouvé.")

    sex = "male" if current_patient.sex == "M" else "female" if current_patient.sex == "F" else "unknown"
    timepoint_info = current_patient.timepoint_infos.get(timepoint)
    age = int(timepoint_info.age) if timepoint_info else "unknown"

    for lesion in lesion_rows:
        lesion_id = lesion["lesion_id"]

        # Intensité et volume
        volume = float(lesion.get("lesion_volume_mm3", 0))
        mean_int = float(lesion.get("lesion_mean", 0))
        min_int = float(lesion.get("lesion_min", 0))
        max_int = float(lesion.get("lesion_max", 0))

        # Contraste
        contrast_val = None
        for c in contrast_rows:
            if (c.get("patient") == patient_id and
                c.get("timepoint") == timepoint and
                c.get("lesion_id") == lesion_id):
                contrast_val = float(c.get("contrast_lesion_neighborhood", 0))
                break
        contrast_str = f"{contrast_val:.2f}" if contrast_val is not None else "unknown"

        # Localisation anatomique
        prop_cols = [k for k in lesion.keys() if k.endswith("_prop")]
        prop_values = [(k, float(lesion[k])) for k in prop_cols if float(lesion[k]) > 0]
        prop_values.sort(key=lambda x: x[1], reverse=True)
        zones_text = [f"{k.replace('_prop','').replace('_',' ')} ({v*100:.1f}%)" for k,v in prop_values]
        zones_str = ", ".join(zones_text)

        # Texte final
        prefixes = [
            f"This {modality} scan corresponds to a {age}-year-old {sex} subject {patient_id} at timepoint {timepoint}. ",
            f"This is a {modality} scan of a {sex} subject {patient_id}, aged {age}, timepoint {timepoint}. ",
            f"This {modality} scan shows a {sex} subject {patient_id} aged {age} of timepoint {timepoint}. ",
            f"A {modality} scan of a {sex} subject {patient_id} at timepoint {timepoint}, {age} years old. ",
            f"Here is a {modality} scan of a {sex} subject {patient_id} at timepoint {timepoint} aged {age}. ",
            f"This {modality} scan captures the brain of a {sex} subject {patient_id} aged {age}, timepoint {timepoint}. ",
            f"A {modality} scan from a {sex} subject {patient_id}, {age} years old, at timepoint {timepoint}. ",
            f"This {modality} scan, taken from a {sex} subject {patient_id} aged {age} at timepoint {timepoint}. ",
            f"This {modality} scan provides a view of the brain of a {age} subject {patient_id}, aged {age}. "
        ]
        lesion_text = prefixes[randint(0, len(prefixes)-1)] + (
            f"Lesion ID {lesion_id} has a volume of {volume:.1f} mm³. "
            f"The lesion shows a mean intensity of {mean_int:.1f}, "
            f"with values ranging from {min_int:.1f} to {max_int:.1f}, "
            f"and a lesion-to-neighborhood contrast of {contrast_str}. "
            f"It is primarily located in {zones_str}."
        )

        # Sauvegarde
        filename = f"{patient_id}_{timepoint}_{modality}_L{lesion_id}_3D_description.txt"
        output_path = template_txt_dir_path / filename
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with output_path.open("w", encoding="utf-8") as f:
            f.write(lesion_text)